
# Local


In [2]:
! pip install --quiet -r ../requirements.txt

import sys
import os
sys.path.append(os.path.expanduser("~/repos/NeurOps/pytorch"))

# Colab


In [3]:
# ! git clone https://github.com/SuReLI/NeurOps.git
# ! git clone https://github.com/k8lion/GrowAndPrune.git
# %cd GrowAndPrune/src
# ! pip install --quiet -r ../requirements.txt

# import sys
# import os
# sys.path.append("../../NeurOps/pytorch")

# Imports

In [14]:
import numpy as np
import torch
import copy
import math
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import HTML
from collections import OrderedDict

from neurops import *

from growprune import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
# !wget -O ../../data/vgg11-8a719046.pth https://download.pytorch.org/models/vgg11-8a719046.pth 

--2023-02-06 15:26:35--  https://download.pytorch.org/models/vgg11-8a719046.pth
Resolving download.pytorch.org (download.pytorch.org)... 18.66.196.80, 18.66.196.101, 18.66.196.22, ...
Connecting to download.pytorch.org (download.pytorch.org)|18.66.196.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 531460341 (507M) [binary/octet-stream]
Saving to: ‘vgg11-8a719046.pth’

vgg11-8a719046.pth  100%[===================>] 506.84M  5.43MB/s    in 3m 11s  

2023-02-06 15:29:46 (2.66 MB/s) - ‘vgg11-8a719046.pth’ saved [531460341/531460341]

--2023-02-06 15:29:46--  http://../data/vgg11-8a719046.pth
Resolving .. (..)... failed: nodename nor servname provided, or not known.
wget: unable to resolve host address ‘..’
FINISHED --2023-02-06 15:29:46--
Total wall clock time: 3m 11s
Downloaded: 1 files, 507M in 3m 11s (2.66 MB/s)


In [29]:
copy_linear = True
limit = 15 if not copy_linear else 19
model = ModVGG11(num_classes=10, avgpooldim=7).to(device)
weights = torch.load("../../data/vgg11-8a719046.pth")
renamed = OrderedDict()
for i, (key, value) in enumerate(zip(model.state_dict(), weights.values())):
    if i <= limit:
        renamed[key] = value
    else:
        renamed[key] = model.state_dict()[key]
model.load_state_dict(renamed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

In [26]:
train_loader, validation_loader, test_loader = get_galaxy10_dataloaders(path_to_dir="../..", batch_size=-1, dim=224)

True True


In [19]:
# test = torch.randn(1, 3, 64, 64).to(device)
# for layer in model:
#     test = layer(test)
#     print(test.shape)

torch.Size([1, 64, 32, 32])
torch.Size([1, 128, 16, 16])
torch.Size([1, 256, 16, 16])
torch.Size([1, 256, 8, 8])
torch.Size([1, 512, 8, 8])
torch.Size([1, 512, 4, 4])
torch.Size([1, 512, 4, 4])
torch.Size([1, 512, 1, 1])
torch.Size([1, 4096])
torch.Size([1, 4096])
torch.Size([1, 10])


In [33]:
train(model, train_loader, optimizer, criterion, epochs=10, val_loader=validation_loader, device=device, verbose=True)

Train Epoch: 0 [0/15963 (0%)]	Loss: 0.815988
Train Epoch: 0 [3200/15963 (22%)]	Loss: 0.958542
Train Epoch: 0 [6400/15963 (45%)]	Loss: 0.934718
Train Epoch: 0 [9600/15963 (67%)]	Loss: 1.289894
Train Epoch: 0 [12800/15963 (89%)]	Loss: 0.643783
Validation: Average loss: 0.0030, Accuracy: 1110/15963 (6.95%)


([0.002968141285539577], [0.06953580154106372], None)